In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import argparse
import torch

from data.data_utils import load_url_dataset

In [3]:
# Load the dataset
num_workers = 2
batch_size = 4
splits_directory = "data/splits"

(trainloader, validloader, testloader), classes = load_url_dataset(splits_directory, batch_size, num_workers=num_workers)

In [4]:
# Load the roberta model
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large')
roberta.eval()

Using cache found in /home/local/USHERBROOKE/levj1404/.cache/torch/hub/pytorch_fairseq_main
INFO:fairseq.tasks.text_to_speech:Please install tensorboardX: pip install tensorboardX
INFO:root:running build_ext
/home/local/USHERBROOKE/levj1404/miniconda3/envs/nlp/lib/python3.10/site-packages/torch/utils/cpp_extension.py:500: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
INFO:root:building 'fairseq.libbleu' extension
INFO:root:creating build
INFO:root:creating build/temp.linux-x86_64-cpython-310
INFO:root:creating build/temp.linux-x86_64-cpython-310/fairseq
INFO:root:creating build/temp.linux-x86_64-cpython-310/fairseq/clib
INFO:root:creating build/temp.linux-x86_64-cpython-310/fairseq/clib/libbleu
INFO:root:gcc -pthread -B /home/local/USHERBROOKE/levj1404/miniconda3/envs/nlp/compiler_compat -Wno-unused-result -Wsign-compare -DNDEBUG -f

Compiling fairseq/data/data_utils_fast.pyx because it changed.
[1/1] Cythonizing fairseq/data/data_utils_fast.pyx


INFO:root:building 'fairseq.data.data_utils_fast' extension
INFO:root:creating build/temp.linux-x86_64-cpython-310/fairseq/data
INFO:root:gcc -pthread -B /home/local/USHERBROOKE/levj1404/miniconda3/envs/nlp/compiler_compat -Wno-unused-result -Wsign-compare -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /home/local/USHERBROOKE/levj1404/miniconda3/envs/nlp/include -fPIC -O2 -isystem /home/local/USHERBROOKE/levj1404/miniconda3/envs/nlp/include -fPIC -I/home/local/USHERBROOKE/levj1404/miniconda3/envs/nlp/lib/python3.10/site-packages/numpy/core/include -I/home/local/USHERBROOKE/levj1404/miniconda3/envs/nlp/lib/python3.10/site-packages/numpy/core/include -I/home/local/USHERBROOKE/levj1404/miniconda3/envs/nlp/include/python3.10 -c fairseq/data/data_utils_fast.cpp -o build/temp.linux-x86_64-cpython-310/fairseq/data/data_utils_fast.o -std=c++11 -O3 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -DTORCH_EXTEN

Compiling fairseq/data/token_block_utils_fast.pyx because it changed.
[1/1] Cythonizing fairseq/data/token_block_utils_fast.pyx


INFO:root:building 'fairseq.data.token_block_utils_fast' extension
INFO:root:gcc -pthread -B /home/local/USHERBROOKE/levj1404/miniconda3/envs/nlp/compiler_compat -Wno-unused-result -Wsign-compare -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /home/local/USHERBROOKE/levj1404/miniconda3/envs/nlp/include -fPIC -O2 -isystem /home/local/USHERBROOKE/levj1404/miniconda3/envs/nlp/include -fPIC -I/home/local/USHERBROOKE/levj1404/miniconda3/envs/nlp/lib/python3.10/site-packages/numpy/core/include -I/home/local/USHERBROOKE/levj1404/miniconda3/envs/nlp/lib/python3.10/site-packages/numpy/core/include -I/home/local/USHERBROOKE/levj1404/miniconda3/envs/nlp/include/python3.10 -c fairseq/data/token_block_utils_fast.cpp -o build/temp.linux-x86_64-cpython-310/fairseq/data/token_block_utils_fast.o -std=c++11 -O3 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -DTORCH_EXTENSION_NAME=token_block_utils_fast -D_GLIBCXX_USE

RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 1024, padding_idx=1)
        (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0-23): 24 x TransformerEncoderLayerBase(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
       

In [6]:
tokens = roberta.encode("Hello, world!")
tokens

tensor([    0, 31414,     6,   232,   328,     2])

In [7]:
roberta.decode(tokens)

'Hello, world!'

In [12]:
urls, labels = next(iter(testloader))

In [14]:
embeddings = roberta.extract_features(urls)

tensor([1, 0, 0, 0])